### Task 3
In this task, I was required to make a new data frame that show us the destination between any given city to Tel Aviv.
To get this information I will use Google's services, geocode & distancematrix.

In [166]:
import requests
import json
import urllib
import pandas as pd

let's check how geocode & distancematrix works and what the data they provide us. 
first<b> geocode: </b>

In [167]:
address = "torunto, canda"
key = open("GOOGLE API KEY.txt", "r")
key = key.read()

In [168]:
url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address,key)

In [169]:
response = requests.get(url)
print(response.status_code)
response = requests.get(url).json()
print(type(response))

200
<class 'dict'>


In [170]:
print(response)

{'results': [{'address_components': [{'long_name': 'Toronto', 'short_name': 'Toronto', 'types': ['locality', 'political']}, {'long_name': 'City Of Toronto', 'short_name': 'City Of Toronto', 'types': ['administrative_area_level_3', 'political']}, {'long_name': 'Toronto', 'short_name': 'Toronto', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'Ontario', 'short_name': 'ON', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'Canada', 'short_name': 'CA', 'types': ['country', 'political']}], 'formatted_address': 'Toronto, ON, Canada', 'geometry': {'bounds': {'northeast': {'lat': 43.8554579, 'lng': -79.1168971}, 'southwest': {'lat': 43.5810245, 'lng': -79.639219}}, 'location': {'lat': 43.653226, 'lng': -79.3831843}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 43.8554579, 'lng': -79.1168971}, 'southwest': {'lat': 43.5810245, 'lng': -79.639219}}}, 'partial_match': True, 'place_id': 'ChIJpTvG15DL1IkRd8S0KlBVNTI', 'types': ['localit

now the <b> distancematrix: </b>

In [171]:
Serviceurl ="https://maps.googleapis.com/maps/api/distancematrix/json?"
parms = dict()
parms["origins"] = "tel aviv"
parms['destinations'] = "ROME"
parms['key'] = key
url = Serviceurl + urllib.parse.urlencode(parms)

In [172]:
response = requests.get(url)
print(response.status_code)
response = requests.get(url).json()
print(type(response))

200
<class 'dict'>


In [173]:
print(response)

{'destination_addresses': ['Rome, Metropolitan City of Rome Capital, Italy'], 'origin_addresses': ['Tel Aviv-Yafo, Israel'], 'rows': [{'elements': [{'distance': {'text': '4,040 km', 'value': 4040107}, 'duration': {'text': '1 day 19 hours', 'value': 155268}, 'status': 'OK'}]}], 'status': 'OK'}


### The function
Now I want to build a function that will help me get the data I need.

 


first i need two importent function, one that changes the duration to hours and second function that extract the Longitude & Latitude 

In [195]:
def To_hours(x):
    x = x.split()
    if x[1] == "day" or x[1] == "days" :
        x = int(x[0])*24 + int(x[2])
        return str(x) + " hours"
    else:
        return ' '.join(x)

In [196]:
To_hours('1 day 19 hours')

'43 hours'

In [197]:
def dist_dur_extract (origins, target):
    Serviceurl ="https://maps.googleapis.com/maps/api/distancematrix/json?"
    parms = dict()
    parms["origins"] = origins
    parms['destinations'] = target
    parms['key'] = key
    url = Serviceurl + urllib.parse.urlencode(parms)
    responseChack = requests.get(url)
    response = requests.get(url).json()
    
    if responseChack.status_code != 200:
        return "error in the Api:" + responseChack.status_code 
    else:
        dist = response["rows"][0]['elements'][0]['distance']['text']
        dur = response['rows'][0]['elements'][0]['duration']['text']
        dur = To_hours(dur)
        return dist , dur

In [198]:
dist_dur_extract("tel aviv","rome")

('4,040 km', '43 hours')

In [199]:
def get_Longitude_Latitude(target):
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (target,key)
    response = requests.get(url)
    if response.status_code != 200:
        return "error in the Api:" + response.status_code 
    else:
        response = requests.get(url).json()
        latitude = response['results'][0]['geometry']['location']['lat']
        longitude = response['results'][0]['geometry']['location']['lng']
        return longitude, latitude 

In [200]:
get_Longitude_Latitude("London")

(-0.1275862, 51.5072178)

### The dataframe


In [201]:
citys = open("dests.txt", "r")
citys = citys.read()
print(type(response))
print(citys)

<class 'dict'>
Istanbul
Amsterdam
Valletta
Basel
Doha


In [202]:
cityList = []
for city in citys.split():
    cityList.append(city)
print(cityList)

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']


In [216]:
dic = {
    "Target": [],
    "Distance_km" :[],
    "Duration (hour+minutes)": [],
    "Longitude" : [],
    "latitude" : []
}


In [217]:
for city in cityList:
    dist, dur = dist_dur_extract("tel aviv", city)
    longitude, latitude = get_Longitude_Latitude(city)
    
    dic["Target"].append(city)
    dic["Distance_km"].append(dist)
    dic["Duration (hour+minutes)"].append(dur)
    dic["Longitude"].append(longitude)
    dic["latitude"].append(latitude)

In [218]:
df = pd.DataFrame(data = dic)
df

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
0,Istanbul,"1,815 km",21 hours 3 mins,28.978359,41.008238
1,Amsterdam,"4,533 km",48 hours,4.904139,52.367573
2,Valletta,"3,793 km",51 hours,14.514100,35.899237
3,Basel,"4,093 km",44 hours,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 mins,51.531040,25.285447


### The 3 cities furthest from Tel Aviv.

In [215]:
df = df.sort_values(by='Distance_km',ascending = False)
df[["Target","Distance_km"]][:3]

,Target,Distance_km
1,Amsterdam,"4,545 km"
3,Basel,"4,093 km"
2,Valletta,"3,793 km"
